In [ ]:
import kagglehub

path = kagglehub.dataset_download("iarunava/cell-images-for-detecting-malaria")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cell-images-for-detecting-malaria


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split, Subset
from torch.optim import Adam
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import os
from tqdm.auto import tqdm, trange

In [ ]:
class ChannelAttention(nn.Module):

  def __init__(self, channels, reduction = 16, dropout = 0.1):
    super().__init__()

    self.avg_pool = nn.AdaptiveAvgPool2d(1)
    self.fc = nn.Sequential(
        nn.Linear(channels, channels // reduction, bias = False),
        nn.Dropout(dropout),
        nn.GLU(),
        nn.Linear(channels // reduction // 2, channels, bias = False),
        nn.Sigmoid()
    )

  def forward(self, x):
    b, c, _, _ = x.size()
    y = self.avg_pool(x).view(b, c)
    y = self.fc(y).view(b,c,1,1)
    return x * y.expand_as(x)

In [ ]:
model = models.resnet18(pretrained = True)
print(model)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 155MB/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
class ResNetWithAttention(nn.Module):

  def __init__(self, num_classes = 2):
    super().__init__()

    self.resnet = models.resnet18(pretrained = True)

    self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    self.gates = nn.ModuleList()

    for layer in [self.resnet.layer1, self.resnet.layer2, self.resnet.layer3, self.resnet.layer4]:
      for block in layer:
        self.gates.append(ChannelAttention(block.conv2.out_channels))


  def forward(self, x):

    x = self.resnet.conv1(x)
    x = self.resnet.bn1(x)
    x = self.resnet.relu(x)
    x = self.resnet.maxpool(x)

    gate_idx = 0

    for layer in [self.resnet.layer1, self.resnet.layer2, self.resnet.layer3, self.resnet.layer4]:
      for block in layer:

        identity = x
        x = block.conv1(x)
        x = block.bn1(x)
        x = block.relu(x)
        x = block.conv2(x)
        x = block.bn2(x)
        x = self.gates[gate_idx](x)

        gate_idx += 1

        if block.downsample is not None:
          identity = block.downsample(identity)

        x += identity
        x = block.relu(x)

    x = self.resnet.avgpool(x)
    x = torch.flatten(x, 1)
    y = self.resnet.fc(x)

    return y

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    preds, targets = [], []

    loop = tqdm(dataloader, desc='Training', leave=True)

    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)
        preds.extend(torch.argmax(outputs, dim=1).cpu().tolist())
        targets.extend(labels.cpu().tolist())

    avg_loss = total_loss / len(dataloader.dataset)
    acc = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds)

    return avg_loss, acc, f1

def eval_epoch(model, dataloader, criterion, device, description="Evaluating"):
    model.eval()
    total_loss = 0
    preds, targets = [], []

    loop = tqdm(dataloader, desc=description, leave=True)

    with torch.no_grad():
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)

            preds.extend(torch.argmax(outputs, dim=1).cpu().tolist())
            targets.extend(labels.cpu().tolist())
            loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(dataloader.dataset)
    acc = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds)

    return avg_loss, acc, f1


In [ ]:
class PlainResNet18(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

In [ ]:
def freeze_layers(num_layers):

  attention_model = ResNetWithAttention(num_classes=2)
  plain_model = PlainResNet18(num_classes=2)

  for model in [attention_model, plain_model]:

    for param in model.resnet.parameters():
      param.requires_grad = False
    for param in model.resnet.fc.parameters():
      param.requires_grad = True

  for param in attention_model.gates.parameters():
    param.requires_grad = True

  for layer in ["layer4", "layer3", "layer2", "layer1"][:num_layers]:
    for model in [attention_model, plain_model]:
      for param in getattr(model.resnet, layer).parameters():
        param.requires_grad = True

  return attention_model, plain_model

In [ ]:
def analyse(train_loader, val_loader, test_loader, unfreeze_layers = 0, epochs = 5):

  criterion = nn.CrossEntropyLoss()
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  attention_model, plain_model = freeze_layers(unfreeze_layers)

  attention_model.to(device)
  plain_model.to(device)

  tqdm.write(f"Number of unfrozen layers: {unfreeze_layers}")

  for model_name, model in [('ResNet18 with Attention', attention_model), ('Plain ResNet18', plain_model)]:
    tqdm.write(f"Training {model_name} model...")

    for epoch in range(epochs):

      optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

      train_loss, train_acc, train_f1 = train_epoch(model, train_loader, criterion, optimizer, device)
      val_loss, val_acc, val_f1 = eval_epoch(model, val_loader, criterion, device)

      tqdm.write(f"Epoch {epoch+1}/{epochs} | "
            f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f} | "
            f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}")

    test_loss, test_acc, test_f1 = eval_epoch(model, test_loader, criterion, device, description="Testing")
    tqdm.write(f"Final Test Loss: {test_loss:.4f}, Acc: {test_acc:.4f}, F1: {test_f1:.4f}")

In [ ]:
path = path + "/cell_images/cell_images"
dataset = datasets.ImageFolder(path)

size = len(dataset)
train_size = int(0.7 * size)
val_size = int(0.15 * size)
test_size = size - train_size - val_size

train_idx, val_idx, test_idx = random_split(range(size), [train_size, val_size, test_size])

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = Subset(datasets.ImageFolder(path, transform=train_transforms), train_idx)
val_dataset = Subset(datasets.ImageFolder(path, transform=test_val_transforms), val_idx)
test_dataset = Subset(datasets.ImageFolder(path, transform=test_val_transforms), test_idx)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
analyse(train_loader, val_loader, test_loader, unfreeze_layers=0, epochs=3)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 97.2MB/s]


Number of unfrozen layers: 0
Training ResNet18 with Attention model...


Training:   0%|          | 0/302 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/65 [00:00<?, ?it/s]

Epoch 1/3 | Train Loss: 0.4705, Acc: 0.7670, F1: 0.7679 | Val Loss: 0.2840, Acc: 0.8955, F1: 0.9008


...

Training:   0%|          | 0/302 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/65 [00:00<?, ?it/s]

Epoch 3/3 | Train Loss: 0.3206, Acc: 0.8759, F1: 0.8769 | Val Loss: 0.3089, Acc: 0.8848, F1: 0.8897


Testing:   0%|          | 0/65 [00:00<?, ?it/s]

Final Test Loss: 0.3215, Acc: 0.8788, F1: 0.8784


In [ ]:
analyse(train_loader, val_loader, test_loader, unfreeze_layers=1, epochs=3)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 42.7MB/s]


Number of unfrozen layers: 1
Training ResNet18 with Attention model...


Training:   0%|          | 0/302 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/65 [00:00<?, ?it/s]

Epoch 1/3 | Train Loss: 0.1501, Acc: 0.9440, F1: 0.9450 | Val Loss: 0.1109, Acc: 0.9620, F1: 0.9624


...

Training:   0%|          | 0/302 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/65 [00:00<?, ?it/s]

Epoch 3/3 | Train Loss: 0.0909, Acc: 0.9668, F1: 0.9672 | Val Loss: 0.1073, Acc: 0.9639, F1: 0.9647


Testing:   0%|          | 0/65 [00:00<?, ?it/s]

Final Test Loss: 0.1147, Acc: 0.9618, F1: 0.9611


In [ ]:
analyse(train_loader, val_loader, test_loader, unfreeze_layers=2, epochs=3)

Number of unfrozen layers: 2
Training ResNet18 with Attention model...


Training:   0%|          | 0/302 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/65 [00:00<?, ?it/s]

Epoch 1/3 | Train Loss: 0.1291, Acc: 0.9532, F1: 0.9534 | Val Loss: 0.1221, Acc: 0.9586, F1: 0.9597


Training:   0%|          | 0/302 [00:00<?, ?it/s]

...

Training:   0%|          | 0/302 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/65 [00:00<?, ?it/s]

Epoch 3/3 | Train Loss: 0.0804, Acc: 0.9712, F1: 0.9713 | Val Loss: 0.0967, Acc: 0.9678, F1: 0.9687


Testing:   0%|          | 0/65 [00:00<?, ?it/s]

Final Test Loss: 0.0934, Acc: 0.9664, F1: 0.9668


In [ ]:
analyse(train_loader, val_loader, test_loader, unfreeze_layers=3, epochs=3)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 138MB/s]


Number of unfrozen layers: 3
Training ResNet18 with Attention model...


Training:   0%|          | 0/302 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/65 [00:00<?, ?it/s]

Epoch 1/3 | Train Loss: 0.1218, Acc: 0.9556, F1: 0.9563 | Val Loss: 0.1069, Acc: 0.9630, F1: 0.9633


...

Training:   0%|          | 0/302 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/65 [00:00<?, ?it/s]

Epoch 3/3 | Train Loss: 0.0747, Acc: 0.9744, F1: 0.9747 | Val Loss: 0.0978, Acc: 0.9642, F1: 0.9648


Testing:   0%|          | 0/65 [00:00<?, ?it/s]

Final Test Loss: 0.0964, Acc: 0.9635, F1: 0.9640


In [ ]:
analyse(train_loader, val_loader, test_loader, unfreeze_layers=4, epochs=3)

Number of unfrozen layers: 4
Training ResNet18 with Attention model...


Training:   0%|          | 0/302 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/65 [00:00<?, ?it/s]

Epoch 1/3 | Train Loss: 0.1235, Acc: 0.9546, F1: 0.9548 | Val Loss: 0.1045, Acc: 0.9627, F1: 0.9638


Training:   0%|          | 0/302 [00:00<?, ?it/s]

...

Training:   0%|          | 0/302 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/65 [00:00<?, ?it/s]

Epoch 3/3 | Train Loss: 0.0781, Acc: 0.9714, F1: 0.9716 | Val Loss: 0.0931, Acc: 0.9690, F1: 0.9695


Testing:   0%|          | 0/65 [00:00<?, ?it/s]

Final Test Loss: 0.0886, Acc: 0.9703, F1: 0.9703
